In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import xgboost as xgb

from joblib import dump, load

pd.set_option('display.max_columns', None)

C:\Users\imacd_0odruq3\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# stats = pd.read_csv('../../data/adjusted_value_models_combined_6_game_rolling.csv')
stats = pd.read_csv('../../pipeline/data/07_adjusted_value_models_aggregated.csv')
schedule = pd.read_csv('../../data/schedule_final.csv')
betting = pd.read_csv('../../data/betting_data_cleaned_with_returns.csv')
betting = betting.drop(columns=['Unnamed: 0'])

stats = stats.dropna()

In [3]:
stats.head()

,Unnamed: 0,season,week,team,qb,passing_value_adjusted,rushing_value_adjusted,total_plays_standardized,total_possession_time_standardized,pass_percentage_standardized,pass_def_value_adjusted,rush_def_value_adjusted,special_teams_value,team_full
64,64,2014,3,ARI,D. Stanton,-0.857878,0.359969,-0.101593,-0.628596,-0.373010,-0.160789,0.607663,1.173452,Arizona Cardinals
65,65,2014,3,ATL,M. Ryan,-0.640304,0.709637,0.279779,-0.357458,0.832298,-0.824354,-0.311896,1.010280,Atlanta Falcons
66,66,2014,3,BAL,J. Flacco,0.388264,0.048372,1.154692,0.569921,-0.103556,0.786102,-0.208942,0.137405,Baltimore Ravens
67,67,2014,3,BUF,E. Manuel,0.531544,-0.912222,-0.370797,-0.198053,-1.418698,0.256764,0.606452,1.562575,Buffalo Bills
68,68,2014,3,CAR,C. Newton,0.604152,-0.878958,-0.101593,-0.226359,0.373782,0.309284,0.344995,1.301579,Carolina Panthers


In [4]:
stats

,Unnamed: 0,season,week,team,qb,passing_value_adjusted,rushing_value_adjusted,total_plays_standardized,total_possession_time_standardized,pass_percentage_standardized,pass_def_value_adjusted,rush_def_value_adjusted,special_teams_value,team_full
64,64,2014,3,ARI,D. Stanton,-0.857878,0.359969,-0.101593,-0.628596,-0.373010,-0.160789,0.607663,1.173452,Arizona Cardinals
65,65,2014,3,ATL,M. Ryan,-0.640304,0.709637,0.279779,-0.357458,0.832298,-0.824354,-0.311896,1.010280,Atlanta Falcons
66,66,2014,3,BAL,J. Flacco,0.388264,0.048372,1.154692,0.569921,-0.103556,0.786102,-0.208942,0.137405,Baltimore Ravens
67,67,2014,3,BUF,E. Manuel,0.531544,-0.912222,-0.370797,-0.198053,-1.418698,0.256764,0.606452,1.562575,Buffalo Bills
68,68,2014,3,CAR,C. Newton,0.604152,-0.878958,-0.101593,-0.226359,0.373782,0.309284,0.344995,1.301579,Carolina Panthers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4856,4856,2022,21,CIN,J. Burrow,0.363199,0.164351,0.520941,0.625788,0.405253,-0.080416,0.555210,0.215284,Cincinnati Bengals
4857,4857,2022,21,KC,P. Mahomes,0.474375,0.512161,-0.449315,-0.141815,-0.018159,0.781146,-0.771093,-0.504759,Kansas City Chiefs
4858,4858,2022,21,PHI,J. Hurts,0.131127,1.616761,0.672368,0.560610,-0.941733,0.742787,-0.131826,0.329906,Philadelphia Eagles
4860,4860,2022,22,KC,P. Mahomes,0.474073,0.212715,-0.185719,0.080905,0.354606,0.612055,-0.822898,-0.455509,Kansas City Chiefs


In [5]:
schedule.head()

,Unnamed: 0,date,season,week,season_type,home,away,home_score,away_score,home_qb,away_qb,home_qb_abv,away_qb_abv
0,0,2014-09-04,2014,1,REG,Seattle Seahawks,Green Bay Packers,36,16,Russell Wilson,Aaron Rodgers,R. Wilson,A. Rodgers
1,1,2014-09-07,2014,1,REG,Baltimore Ravens,Cincinnati Bengals,16,23,Joe Flacco,Andy Dalton,J. Flacco,A. Dalton
2,2,2014-09-07,2014,1,REG,Houston Texans,Washington Football Team,17,6,Ryan Fitzpatrick,Robert Griffin,R. Fitzpatrick,R. Griffin
3,3,2014-09-07,2014,1,REG,Chicago Bears,Buffalo Bills,20,23,Jay Cutler,EJ Manuel,J. Cutler,E. Manuel
4,4,2014-09-07,2014,1,REG,Pittsburgh Steelers,Cleveland Browns,30,27,Ben Roethlisberger,Brian Hoyer,B. Roethlisberger,B. Hoyer


In [6]:
print(len(stats))
print(len(schedule))

4555
2438


In [8]:
nfl = schedule.copy()
nfl = nfl.merge(stats.add_suffix('_home'), how='left', left_on = ['season', 'week', 'home', 'home_qb_abv'], 
                right_on = ['season_home', 'week_home', 'team_full_home', 'qb_home'])
nfl = nfl.merge(stats.add_suffix('_away'), how='left', left_on = ['season', 'week', 'away', 'away_qb_abv'], 
                right_on = ['season_away', 'week_away', 'team_full_away', 'qb_away'])
# nfl = nfl[['date', 'season', 'week', 'season_type', 'home', 'away', 'home_score', 'away_score', 'home_qb', 'away_qb',
#           'qb_adjusted_value_home', 'rushing_adjusted_value_home', 'qb_rushing_value_pct_home', 'qb_def_adjusted_value_home', 'rush_def_adjusted_value_home', 'special_teams_value_home',
#           'qb_adjusted_value_away', 'rushing_adjusted_value_away', 'qb_rushing_value_pct_away', 'qb_def_adjusted_value_away', 'rush_def_adjusted_value_away', 'special_teams_value_away']]
# nfl.head()
nfl = nfl[['date', 'season', 'week', 'season_type', 'home', 'away', 'home_score', 'away_score', 'home_qb', 'away_qb',
          'passing_value_adjusted_home', 'rushing_value_adjusted_home', 'pass_def_value_adjusted_home', 'rush_def_value_adjusted_home', 'special_teams_value_home',
          'passing_value_adjusted_away', 'rushing_value_adjusted_away', 'pass_def_value_adjusted_away', 'rush_def_value_adjusted_away', 'special_teams_value_away',
          'total_possession_time_standardized_home', 'total_possession_time_standardized_away', 
          'total_plays_standardized_home', 'total_plays_standardized_away',
          'pass_percentage_standardized_home', 'pass_percentage_standardized_away']]
nfl.head()

,date,season,week,season_type,home,away,home_score,away_score,home_qb,away_qb,passing_value_adjusted_home,rushing_value_adjusted_home,pass_def_value_adjusted_home,rush_def_value_adjusted_home,special_teams_value_home,passing_value_adjusted_away,rushing_value_adjusted_away,pass_def_value_adjusted_away,rush_def_value_adjusted_away,special_teams_value_away,total_possession_time_standardized_home,total_possession_time_standardized_away,total_plays_standardized_home,total_plays_standardized_away,pass_percentage_standardized_home,pass_percentage_standardized_away
0,2014-09-04,2014,1,REG,Seattle Seahawks,Green Bay Packers,36,16,Russell Wilson,Aaron Rodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-09-07,2014,1,REG,Baltimore Ravens,Cincinnati Bengals,16,23,Joe Flacco,Andy Dalton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-09-07,2014,1,REG,Houston Texans,Washington Football Team,17,6,Ryan Fitzpatrick,Robert Griffin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-09-07,2014,1,REG,Chicago Bears,Buffalo Bills,20,23,Jay Cutler,EJ Manuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-09-07,2014,1,REG,Pittsburgh Steelers,Cleveland Browns,30,27,Ben Roethlisberger,Brian Hoyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
print(len(nfl))

2438


In [10]:
betting.head()

,date,home_team,away_team,home_score,away_score,total_score_actual,home_moneyline,away_moneyline,home_spread,away_spread,total_score_line,over_under_result,home_team_actual_line,away_team_actual_line,spread_cover_result,home_implied_prob,away_implied_prob,game_winner,favorite_return,underdog_return,home_return,away_return,favorite_spread_return,underdog_spread_return,home_spread_return,away_spread_return,over_return,under_return
0,2023-02-12,Philadelphia Eagles,Kansas City Chiefs,35,38,73,-120.0,100.0,-1.0,1.0,51.5,over,3,-3,away,0.545455,0.500000,away,-100.00,100.0,-100.00,100.0,-100.00,90.91,-100.00,90.91,90.91,-100.00
1,2023-01-29,Kansas City Chiefs,Cincinnati Bengals,23,20,43,-130.0,110.0,-2.0,2.0,48.5,under,-3,3,home,0.565217,0.476190,home,76.92,-100.0,76.92,-100.0,90.91,-100.00,90.91,-100.00,-100.00,90.91
2,2023-01-29,Philadelphia Eagles,San Francisco 49ers,31,7,38,-155.0,135.0,-3.0,3.0,45.0,under,-24,24,home,0.607843,0.425532,home,64.52,-100.0,64.52,-100.0,90.91,-100.00,90.91,-100.00,-100.00,90.91
3,2023-01-22,San Francisco 49ers,Dallas Cowboys,19,12,31,-200.0,170.0,-3.5,3.5,46.5,under,-7,7,home,0.666667,0.370370,home,50.00,-100.0,50.00,-100.0,90.91,-100.00,90.91,-100.00,-100.00,90.91
4,2023-01-22,Buffalo Bills,Cincinnati Bengals,10,27,37,-260.0,215.0,-6.0,6.0,48.5,under,17,-17,away,0.722222,0.317460,away,-100.00,215.0,-100.00,215.0,-100.00,90.91,-100.00,90.91,-100.00,90.91


In [11]:
nfl.head()

,date,season,week,season_type,home,away,home_score,away_score,home_qb,away_qb,passing_value_adjusted_home,rushing_value_adjusted_home,pass_def_value_adjusted_home,rush_def_value_adjusted_home,special_teams_value_home,passing_value_adjusted_away,rushing_value_adjusted_away,pass_def_value_adjusted_away,rush_def_value_adjusted_away,special_teams_value_away,total_possession_time_standardized_home,total_possession_time_standardized_away,total_plays_standardized_home,total_plays_standardized_away,pass_percentage_standardized_home,pass_percentage_standardized_away
0,2014-09-04,2014,1,REG,Seattle Seahawks,Green Bay Packers,36,16,Russell Wilson,Aaron Rodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-09-07,2014,1,REG,Baltimore Ravens,Cincinnati Bengals,16,23,Joe Flacco,Andy Dalton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-09-07,2014,1,REG,Houston Texans,Washington Football Team,17,6,Ryan Fitzpatrick,Robert Griffin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-09-07,2014,1,REG,Chicago Bears,Buffalo Bills,20,23,Jay Cutler,EJ Manuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-09-07,2014,1,REG,Pittsburgh Steelers,Cleveland Browns,30,27,Ben Roethlisberger,Brian Hoyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Join in betting data to get point spreads

nfl_cleaned_with_betting = nfl.merge(betting, left_on=['date', 'home', 'away'],
                                            right_on=['date', 'home_team','away_team'])

In [13]:
# Remove any pushes, add binary label for home cover

nfl_cleaned_with_betting_final = nfl_cleaned_with_betting.copy()
nfl_cleaned_with_betting_final['over_hits'] = nfl_cleaned_with_betting_final.apply(
    lambda x: 1 if x.over_under_result == 'over' else 0, axis=1)
nfl_cleaned_with_betting_final = nfl_cleaned_with_betting_final[nfl_cleaned_with_betting_final.over_under_result != 'push']
nfl_cleaned_with_betting_final

,date,season,week,season_type,home,away,home_score_x,away_score_x,home_qb,away_qb,passing_value_adjusted_home,rushing_value_adjusted_home,pass_def_value_adjusted_home,rush_def_value_adjusted_home,special_teams_value_home,passing_value_adjusted_away,rushing_value_adjusted_away,pass_def_value_adjusted_away,rush_def_value_adjusted_away,special_teams_value_away,total_possession_time_standardized_home,total_possession_time_standardized_away,total_plays_standardized_home,total_plays_standardized_away,pass_percentage_standardized_home,pass_percentage_standardized_away,home_team,away_team,home_score_y,away_score_y,total_score_actual,home_moneyline,away_moneyline,home_spread,away_spread,total_score_line,over_under_result,home_team_actual_line,away_team_actual_line,spread_cover_result,home_implied_prob,away_implied_prob,game_winner,favorite_return,underdog_return,home_return,away_return,favorite_spread_return,underdog_spread_return,home_spread_return,away_spread_return,over_return,under_return,over_hits
0,2014-09-04,2014,1,REG,Seattle Seahawks,Green Bay Packers,36,16,Russell Wilson,Aaron Rodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seattle Seahawks,Green Bay Packers,36,16,52,-230.0,205.0,-4.5,4.5,46.5,over,-20,20,home,0.696970,0.327869,home,43.48,-100.0,43.48,-100.0,90.91,-100.00,90.91,-100.00,90.91,-100.00,1
1,2014-09-07,2014,1,REG,Baltimore Ravens,Cincinnati Bengals,16,23,Joe Flacco,Andy Dalton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Baltimore Ravens,Cincinnati Bengals,16,23,39,-113.0,102.0,-1.0,1.0,43.5,under,7,-7,away,0.530516,0.495050,away,-100.00,102.0,-100.00,102.0,-100.00,90.91,-100.00,90.91,-100.00,90.91,0
2,2014-09-07,2014,1,REG,Houston Texans,Washington Football Team,17,6,Ryan Fitzpatrick,Robert Griffin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houston Texans,Washington Football Team,17,6,23,-177.0,159.0,-3.0,3.0,43.5,under,-11,11,home,0.638989,0.386100,home,56.50,-100.0,56.50,-100.0,90.91,-100.00,90.91,-100.00,-100.00,90.91,0
3,2014-09-07,2014,1,REG,Chicago Bears,Buffalo Bills,20,23,Jay Cutler,EJ Manuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chicago Bears,Buffalo Bills,20,23,43,-270.0,239.0,-7.0,7.0,47.0,under,3,-3,away,0.729730,0.294985,away,-100.00,239.0,-100.00,239.0,-100.00,90.91,-100.00,90.91,-100.00,90.91,0
4,2014-09-07,2014,1,REG,Pittsburgh Steelers,Cleveland Browns,30,27,Ben Roethlisberger,Brian Hoyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pittsburgh Steelers,Cleveland Browns,30,27,57,-242.0,216.0,-5.5,5.5,41.5,over,-3,3,away,0.707602,0.316456,home,41.32,-100.0,41.32,-100.0,-100.00,90.91,-100.00,90.91,90.91,-100.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402,2023-01-22,2022,20,POST,Buffalo Bills,Cincinnati Bengals,10,27,Josh Allen,Joe Burrow,-0.151187,0.650933,0.483813,0.589479,-0.215688,0.002339,-0.152032,-0.140611,1.049708,0.204819,-0.153360,0.421317,-0.011858,0.274171,-0.007230,0.889427,Buffalo Bills,Cincinnati Bengals,10,27,37,-260.0,215.0,-6.0,6.0,48.5,under,17,-17,away,0.722222,0.317460,away,-100.00,215.0,-100.00,215.0,-100.00,90.91,-100.00,90.91,-100.00,90.91,0
2403,2023-01-22,2022,20,POST,San Francisco 49ers,Dallas Cowboys,19,12,Brock Purdy,Dak Prescott,1.271880,0.634522,0.303955,0.348498,0.331705,0.563132,-0.743069,-0.104332,0.297954,-0.500316,0.342732,0.301391,-0.247412,0.812579,-0.969892,-0.382391,San Francisco 49ers,Dallas Cowboys,19,12,31,-200.0,170.0,-3.5,3.5,46.5,under,-7,7,home,0.666667,0.370370,home,50.00,-100.0,50.00,-100.0,90.91,-100.00,90.91,-100.00,-100.00,90.91,0
2404,2023-01-29,2022,21,POST,Philadelphia Eagles,San Francisco 49ers,31,7,Jalen Hurts,Brock Purdy,0.131127,1.616761,0.742787,-0.131826,0.329906,NaN,NaN,NaN,NaN,NaN,0.560610,NaN,0.672368,NaN,-0.941733,NaN,Philadelphia Eagles,San Francisc

In [14]:
nfl_cleaned_with_betting_final.to_csv('../../data/adjusted_stats_over_under_modeling_base_df.csv')

In [15]:
# Split data into train and test sets, cross validation for hyperparameter tuning

train_df = nfl_cleaned_with_betting_final[nfl_cleaned_with_betting_final.season <= 2021]
train_df = train_df.dropna()

test_df = nfl_cleaned_with_betting_final[nfl_cleaned_with_betting_final.season == 2022]
test_df = test_df.dropna()

In [16]:
# NOTE
# Dont know if i need to balance this data set at all, point spread betting should be pretty 50/50


# Balance training data set

# Note: Can't balance data set and then do Cross Validation due to data leakage

# Fully balancing the data set doesnt work as well for prediction as the unbalanced, likely because the 
# home team does actually have an important advantage. But, maybe balancing the data set slightly will help

# balance_n = int(np.round((sum(train_df.home_win==1) - sum(train_df.home_win==0))/1.5))
# to_add = train_df[train_df.home_win==0].sample(n=balance_n, random_state=57, replace=True)

# train_df_balanced = pd.concat([train_df, to_add])
# train_df_balanced

In [17]:
# Shuffle training set before cross validation

train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,date,season,week,season_type,home,away,home_score_x,away_score_x,home_qb,away_qb,passing_value_adjusted_home,rushing_value_adjusted_home,pass_def_value_adjusted_home,rush_def_value_adjusted_home,special_teams_value_home,passing_value_adjusted_away,rushing_value_adjusted_away,pass_def_value_adjusted_away,rush_def_value_adjusted_away,special_teams_value_away,total_possession_time_standardized_home,total_possession_time_standardized_away,total_plays_standardized_home,total_plays_standardized_away,pass_percentage_standardized_home,pass_percentage_standardized_away,home_team,away_team,home_score_y,away_score_y,total_score_actual,home_moneyline,away_moneyline,home_spread,away_spread,total_score_line,over_under_result,home_team_actual_line,away_team_actual_line,spread_cover_result,home_implied_prob,away_implied_prob,game_winner,favorite_return,underdog_return,home_return,away_return,favorite_spread_return,underdog_spread_return,home_spread_return,away_spread_return,over_return,under_return,over_hits
1150,2018-10-28,2018,8,REG,Chicago Bears,New York Jets,24,10,Mitchell Trubisky,Sam Darnold,0.423646,0.631278,-0.174603,0.598775,-0.510585,-0.636737,-0.232547,-0.056198,0.338129,0.338709,0.306046,-0.164534,0.425598,0.145177,-0.147265,-0.261253,Chicago Bears,New York Jets,24,10,34,-435.0,325.0,-8.5,8.5,42.5,under,-14,14,home,0.813084,0.235294,home,22.99,-100.0,22.99,-100.00,90.91,-100.00,90.91,-100.00,-100.00,90.91,0
2064,2021-12-25,2021,16,REG,Green Bay Packers,Cleveland Browns,24,22,Aaron Rodgers,Baker Mayfield,1.233658,0.548593,0.066080,-0.791988,-0.966252,-0.402719,-0.043518,0.175581,-0.310391,-1.261027,0.916106,0.024667,0.262954,-0.135243,0.257956,-0.067848,Green Bay Packers,Cleveland Browns,24,22,46,-435.0,325.0,-7.5,7.5,46.5,under,-2,2,away,0.813084,0.235294,home,22.99,-100.0,22.99,-100.00,-100.00,90.91,-100.00,90.91,-100.00,90.91,0
936,2017-11-26,2017,12,REG,Cincinnati Bengals,Cleveland Browns,30,16,Andy Dalton,DeShone Kizer,-0.028467,-1.001899,0.193932,0.427619,-0.739475,-1.525351,0.261091,0.271681,0.811267,-0.275233,-1.669943,-0.676827,-1.868244,-0.152069,0.172185,0.452313,Cincinnati Bengals,Cleveland Browns,30,16,46,-326.0,285.0,-7.5,7.5,39.0,over,-14,14,home,0.765258,0.259740,home,30.67,-100.0,30.67,-100.00,90.91,-100.00,90.91,-100.00,90.91,-100.00,1
216,2014-12-20,2014,16,REG,San Francisco 49ers,Los Angeles Chargers,35,38,Colin Kaepernick,Philip Rivers,-0.448572,-0.374572,-0.233069,0.070572,-0.686801,-0.230551,-0.189492,-0.434935,0.981958,-1.220347,-0.318352,-0.458018,-0.892379,-0.421273,-0.314230,0.776920,San Francisco 49ers,Los Angeles Chargers,35,38,73,-128.0,116.0,-2.0,2.0,40.5,over,3,-3,away,0.561404,0.462963,away,-100.00,116.0,-100.00,116.00,-100.00,90.91,-100.00,90.91,90.91,-100.00,1
1491,2019-12-01,2019,13,REG,Arizona Cardinals,Los Angeles Rams,7,34,Kyler Murray,Jared Goff,-0.096981,0.381885,-0.537923,0.343440,-0.095490,0.139474,-0.741993,-0.567378,-0.064543,0.561093,-0.773103,-1.078505,-0.628784,-0.892379,0.462515,0.298011,Arizona Cardinals,Los Angeles Rams,7,34,41,120.0,-141.0,2.5,-2.5,47.0,under,27,-27,away,0.454545,0.585062,away,70.92,-100.0,-100.00,70.92,90.91,-100.00,-100.00,90.91,-100.00,90.91,0


In [19]:
# Prepare feature list

# feature_list = ['passing_value_home', 'rushing_value_home', 'pass_def_value_home', 'rush_def_value_home',
#                    'passing_value_away', 'rushing_value_away', 'pass_def_value_away', 'rush_def_value_away']

# feature_list = ['qb_value_home', 'passing_value_home', 'rushing_value_home', 'qb_def_value_home', 'pass_def_value_home', 'rush_def_value_home', 'special_teams_value_home',
#                    'qb_value_away', 'passing_value_away', 'rushing_value_away', 'qb_def_value_away', 'pass_def_value_away', 'rush_def_value_away', 'special_teams_value_away']

# feature_list = ['qb_adjusted_value_home', 'rushing_adjusted_value_home', 'qb_def_adjusted_value_home', 'rush_def_adjusted_value_home', 'special_teams_value_home',
#                    'qb_adjusted_value_away', 'rushing_adjusted_value_away', 'qb_def_adjusted_value_away', 'rush_def_adjusted_value_away', 'special_teams_value_away',
#                 'total_score_line']

feature_list = ['passing_value_adjusted_home', 'rushing_value_adjusted_home', 'pass_def_value_adjusted_home', 'rush_def_value_adjusted_home', 'special_teams_value_home',
          'passing_value_adjusted_away', 'rushing_value_adjusted_away', 'pass_def_value_adjusted_away', 'rush_def_value_adjusted_away', 'special_teams_value_away',
          'total_possession_time_standardized_home', 'total_possession_time_standardized_away', 
          'total_plays_standardized_home', 'total_plays_standardized_away',
          'pass_percentage_standardized_home', 'pass_percentage_standardized_away']

In [20]:
# Get features, labels for train, val, and test sets

# train_x = train_df_balanced[feature_list].to_numpy()
# train_y = train_df_balanced.home_win.to_numpy()

train_x = train_df_shuffled[feature_list].to_numpy()
train_y = train_df_shuffled.over_hits.to_numpy()

test_x = test_df[feature_list]
test_y = test_df.over_hits.to_numpy()

In [21]:
train_df_shuffled.week

1150     8
2064    16
936     12
216     16
1491    13
        ..
1327     2
1515    15
1021    17
1727    11
1323     2
Name: week, Length: 1801, dtype: int64

In [22]:
# Function to get predictions and probabilities for train, val, test sets

def get_preds(model, train_x, test_x):
    train_preds = model.predict(train_x)
    train_probs = model.predict_proba(train_x)
    
    test_preds = model.predict(test_x)
    test_probs = model.predict_proba(test_x)
    
    return train_preds, train_probs, test_preds, test_probs

# Function to get accuracy scores for train, val sets

def print_cv_results(model, train_x, train_y, cv_folds, verbose=True):
    cv_results = cross_validate(model, train_x, train_y, cv=5, return_train_score=True)
    
    train_scores = cv_results['train_score']
    val_scores = cv_results['test_score']
    
    if verbose:      
        for i, scores in enumerate(zip(train_scores, val_scores)):
            print('Fold {}, Train Accuracy: {}, Validation Accuracy: {}'.format(i+1,scores[0], scores[1]))     
        print()
        
    print('Average Training Accuracy: {}'.format(np.mean(train_scores)))
    print('Average Validation Accuracy: {}'.format(np.mean(val_scores)))
    
    return np.mean(train_scores), np.mean(val_scores)

In [23]:
# Simple Logistic Regression Model

lr_model = LogisticRegression()

print_cv_results(lr_model, train_x, train_y, 5)

Fold 1, Train Accuracy: 0.5263888888888889, Validation Accuracy: 0.5484764542936288
Fold 2, Train Accuracy: 0.5419847328244275, Validation Accuracy: 0.49722222222222223
Fold 3, Train Accuracy: 0.5655794587092297, Validation Accuracy: 0.49166666666666664
Fold 4, Train Accuracy: 0.5399028452463567, Validation Accuracy: 0.45555555555555555
Fold 5, Train Accuracy: 0.546148507980569, Validation Accuracy: 0.5416666666666666

Average Training Accuracy: 0.5440008867298942
Average Validation Accuracy: 0.506917513080948


(0.5440008867298942, 0.506917513080948)

In [24]:
# Default Random Forest Classifier with no hyperparameter tuning

rf_model = RandomForestClassifier()

print_cv_results(rf_model, train_x, train_y, 5)

Fold 1, Train Accuracy: 1.0, Validation Accuracy: 0.5235457063711911
Fold 2, Train Accuracy: 1.0, Validation Accuracy: 0.5361111111111111
Fold 3, Train Accuracy: 1.0, Validation Accuracy: 0.5083333333333333
Fold 4, Train Accuracy: 1.0, Validation Accuracy: 0.5
Fold 5, Train Accuracy: 1.0, Validation Accuracy: 0.5444444444444444

Average Training Accuracy: 1.0
Average Validation Accuracy: 0.5224869190520159


(1.0, 0.5224869190520159)

In [25]:
# Default XGBoost Model with no hyperparameter tuning

xgb_model = xgb.XGBClassifier()

print_cv_results(xgb_model, train_x, train_y, 5)

C:\Users\imacd_0odruq3\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:07:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\imacd_0odruq3\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:07:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\imacd_0odruq3\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\imacd_0odruq3\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:07:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\imacd_0odruq3\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:07:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold 1, Train Accuracy: 1.0, Validation Accuracy: 0.5734072022160664
Fold 2, Train Accuracy: 1.0, Validation Accuracy: 0.48333333333333334
Fold 3, Train Accuracy: 1.0, Validation Accuracy: 0.4861111111111111
Fold 4, Train Accuracy: 1.0, Validation Accuracy: 0.4777777777777778
Fold 5, Train Accuracy: 1.0, Validation Accuracy: 0.5277777777777778

Average Training Accuracy: 1.0
Average Validation Accuracy: 0.5096814404432133


(1.0, 0.5096814404432133)

In [26]:
# Logistic Regression, with hyperparameter tuning

def logistic_regression_tuning(train_x, train_y, verbose=True):
    
    lr_model = LogisticRegression()
    
    # Hyperparameters to tune
    penalty_list = ['none', 'l2', 'l1', 'elasticnet']
    c_values = [100, 10, 1.0, 0.1, 0.01]
    solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    
    params = dict(solver=solvers, penalty=penalty_list, C=c_values)
    
    clf = GridSearchCV(lr_model, params, return_train_score=True).fit(train_x, train_y)
    
    print('Best Result: {}'.format(clf.best_score_))
    print('Best Parameters: {}'.format(clf.best_params_))
    print()
    
    train_scores = clf.cv_results_['mean_train_score']
    val_scores = clf.cv_results_['mean_test_score']
    param_list = clf.cv_results_['params']
    
    if verbose:
        print('Parameter Combinations and Results:')
        for train_score, val_score, params in zip(train_scores, val_scores, param_list):
            print('Train Score: {}, Val Score: {}, Parameters: {}'.format(train_score, val_score, params))

In [27]:
# Can uncomment and run below to see, but hyperparameter tuning didnt change much. Can probably just roll with the 
# default logistic regression

# logistic_regression_tuning(train_x, train_y)

In [2]:
# Save best logistic regression model 

best_lr_model = LogisticRegression().fit(train_x, train_y)
dump(best_lr_model, 'saved_models/ou_logistic_regression_av.joblib')

NameError: name 'LogisticRegression' is not defined

In [29]:
# Save best random forest model 

best_rf_model = RandomForestClassifier().fit(train_x, train_y)
dump(best_rf_model, 'saved_models/ou_random_forest_av.joblib')

['saved_models/ou_random_forest_av.joblib']

In [30]:
# Save best xg boost model 

best_xgb_model = xgb.XGBClassifier().fit(train_x, train_y)
dump(best_xgb_model, 'saved_models/ou_xg_boost_av.joblib')

C:\Users\imacd_0odruq3\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:07:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


['saved_models/ou_xg_boost_av.joblib']